In [1]:
"""Post process aggregations from EE and combine with other datasets.
-------------------------------------------------------------------------------

combines the different datasets into one result table. 

indicator weights 
bws withdrawal per sector
bwd withdrawal per sector
iav withdrawal per sector
sev withdrawal per sector


Author: Rutger Hofste
Date: 20190128
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04

"""

TESTING = 0
SCRIPT_NAME = "Y2019M01D28_RH_GA_Zonal_Stats_Table_V01"
OUTPUT_VERSION = 6

GCS_INPUT_PATH = "gs://aqueduct30_v01/Y2019M01D17_RH_GA_Zonal_Stats_Weighted_Indicators_EE_V01/output_V05"

BQ_PROJECT_ID = "aqueduct30"
BQ_DATASET_NAME = "aqueduct30v01"
BQ_INPUT_TABLE_NAME_LABEL = "y2018m12d04_rh_master_merge_rawdata_gpd_v02_v05"
BQ_INPUT_TABLE_NAME_GADM  = "y2018m11d12_rh_gadm36_level1_rds_to_bq_v01_v01"
BQ_OUTPUT_TABLE_NAME = "{}_v{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION).lower()


ec2_input_path = "/volumes/data/{}/input_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION) 
ec2_output_path = "/volumes/data/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION) 

s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

print("GCS_INPUT_PATH: " + GCS_INPUT_PATH +
      "\nec2_input_path: " +  ec2_input_path + 
      "\nec2_output_path: " + ec2_output_path + 
      "\ns3_output_path: " + s3_output_path  )



GCS_INPUT_PATH: gs://aqueduct30_v01/Y2019M01D17_RH_GA_Zonal_Stats_Weighted_Indicators_EE_V01/output_V05
ec2_input_path: /volumes/data/Y2019M01D28_RH_GA_Zonal_Stats_Table_V01/input_V06
ec2_output_path: /volumes/data/Y2019M01D28_RH_GA_Zonal_Stats_Table_V01/output_V06
s3_output_path: s3://wri-projects/Aqueduct30/processData/Y2019M01D28_RH_GA_Zonal_Stats_Table_V01/output_V06/


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2019M01D30 UTC 16:34


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
!mkdir -p {ec2_input_path}
!mkdir -p {ec2_output_path}

In [4]:
!gsutil -m cp {GCS_INPUT_PATH}/* {ec2_input_path}

Copying gs://aqueduct30_v01/Y2019M01D17_RH_GA_Zonal_Stats_Weighted_Indicators_EE_V01/output_V05/Dom_weighted_bwd_sumee_export.csv...
Copying gs://aqueduct30_v01/Y2019M01D17_RH_GA_Zonal_Stats_Weighted_Indicators_EE_V01/output_V05/Dom_weighted_bws_sumee_export.csv...
Copying gs://aqueduct30_v01/Y2019M01D17_RH_GA_Zonal_Stats_Weighted_Indicators_EE_V01/output_V05/Dom_weighted_cep_sumee_export.csv...
Copying gs://aqueduct30_v01/Y2019M01D17_RH_GA_Zonal_Stats_Weighted_Indicators_EE_V01/output_V05/Dom_weighted_cfr_sumee_export.csv...
Copying gs://aqueduct30_v01/Y2019M01D17_RH_GA_Zonal_Stats_Weighted_Indicators_EE_V01/output_V05/Dom_weighted_gtd_sumee_export.csv...
Copying gs://aqueduct30_v01/Y2019M01D17_RH_GA_Zonal_Stats_Weighted_Indicators_EE_V01/output_V05/Dom_weighted_drr_sumee_export.csv...
Copying gs://aqueduct30_v01/Y2019M01D17_RH_GA_Zonal_Stats_Weighted_Indicators_EE_V01/output_V05/Dom_weighted_rri_sumee_export.csv...
Copying gs://aqueduct30_v01/Y2019M01D17_RH_GA_Zonal_Stats_Weighted_In

/ [70/70 files][ 11.0 MiB/ 11.0 MiB] 100% Done                                  
Operation completed over 70 objects/11.0 MiB.                                    


In [5]:
import os
import numpy as np
import pandas as pd
from google.cloud import bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/.google.json"
os.environ["GOOGLE_CLOUD_PROJECT"] = "aqueduct30"
client = bigquery.Client(project=BQ_PROJECT_ID)

## Labels

In [6]:
sql = """
SELECT
  indicator,
  AVG(cat) AS cat,
  label
FROM
  `{}.{}.{}`
GROUP BY
  label, indicator
ORDER BY
  indicator, cat
""".format(BQ_PROJECT_ID,BQ_DATASET_NAME,BQ_INPUT_TABLE_NAME_LABEL)

In [7]:
df_labels = pd.read_gbq(query=sql,
                        project_id =BQ_PROJECT_ID,
                        dialect="standard")

In [8]:
df_labels.head()

,indicator,cat,label
0,None,NaN,None
1,bwd,NaN,None
2,bwd,-1.0,Arid and Low Water Use
3,bwd,0.0,Low
4,bwd,1.0,Low - Medium


## GADM Level 1 names

In [9]:
sql = """
SELECT
  gid_1,
  gid_0,
  name_1,
  name_0
FROM
  `{}.{}.{}`
ORDER BY
  gid_1
""".format(BQ_PROJECT_ID,BQ_DATASET_NAME,BQ_INPUT_TABLE_NAME_GADM)

In [10]:
df_gadm_1 = pd.read_gbq(query=sql,
                        project_id =BQ_PROJECT_ID,
                        dialect="standard")

## GADM Level 0 names

In [11]:
sql = """
SELECT
  name_0,
  ANY_VALUE(gid_0) as gid_0
FROM
  `{}.{}.{}`
GROUP BY
  name_0
ORDER BY
  name_0
""".format(BQ_PROJECT_ID,BQ_DATASET_NAME,BQ_INPUT_TABLE_NAME_GADM)

In [12]:
df_gadm_0 = pd.read_gbq(query=sql,
                       project_id =BQ_PROJECT_ID,
                       dialect="standard")

## BWS, BWD, IAV, SEV

In [13]:
def score_to_category(score):
    if np.isnan(score):
        cat = np.nan
    else:
        if score < 5:
            cat = int(np.floor(score))
        else:
            cat = 4
    return cat

In [14]:
def get_weights_df(sector):
    """ Get Dataframe Per sector
    
    """
    input_file_name = "{}_weights_sumee_export.csv".format(sector)
    input_file_path = "{}/{}".format(ec2_input_path,input_file_name)
    df_weights = pd.read_csv(input_file_path)
    df_weights.drop(columns=["system:index",".geo"],
                    inplace=True)
    df_weights.rename(columns={"sum":"sum_weights"},
                      inplace=True)
    return df_weights

def get_weighted_indicator_df(indicator):
    """ Get DataFrame per indicator

    """
    input_file_name = "{}_weighted_{}_sumee_export.csv".format(sector,indicator)
    input_file_path = "{}/{}".format(ec2_input_path,input_file_name)
    df = pd.read_csv(input_file_path)
    df.drop(columns=["system:index",".geo"],
            inplace=True)
    df.rename(columns={"sum":"sum_weighted_indicator"},inplace=True)
    df["indicator_name"] = indicator
    df["sector"] = sector
    return df
    

def province_to_country(df,sector,indicator):
    """ Convert province level dataframe to country level
    DataFrame
    
    
    """   
    df["gid_0"] = df["gid_1"].apply(lambda x:  x.split(".")[0])
    
    grouped = df.groupby('gid_0')
    df_country = df.groupby(by="gid_0",as_index=False).sum()
    df_country["indicator_name"] = indicator
    df_country["sector"] = sector
    return df_country

def process_df(df):
    """ Calculate Score, add cat and label. 
    
    Due to the zonal statistics in Google Earth Engine, 
    some semi masked cells produce score higher than 5. 
    Clipping all scores above 5 to 5. 
    
    Sorts columns alphabetically.
    
    Ranks based on score. Uses minimum rank:
    http://www.datasciencemadesimple.com/rank-dataframe-python-pandas-min-max-dense-rank-group/
    
    
    """
    df["score"]  = df["sum_weighted_indicator"] / df["sum_weights"]
    df["score"].clip(lower=None,upper=5,inplace=True)
    df["cat"] = df["score"].apply(score_to_category)
    df = df.reindex(sorted(df.columns), axis=1)
    df = pd.merge(left=df,
                   right=df_labels,
                   how="left",
                   left_on=["indicator_name","cat"],
                   right_on=["indicator","cat"])
    df.drop(columns=["indicator"],
            inplace=True)
    df["score_ranked"] = df["score"].rank(ascending=False,method="min")
    return df

def  export_df(df,geographic_scale):
    """ Export Dataframe as csv on e2
    and table on BigQuery
    
    Args:
        df(pd.DataFrame)  :  DataFrame to export.
        geographic_scale : "country" or "province"
    
    
    """
    
    if geographic_scale == "country":
        df_merged = pd.merge(left=df,
                             right=df_gadm_0,
                             how="left",
                             left_on="gid_0",
                             right_on="gid_0")
        df_merged = df_merged.reindex(sorted(df_merged.columns), axis=1)
        df_merged_csv = df_merged.set_index("gid_0")
    elif geographic_scale == "province":
        df.drop(columns=["gid_0"],inplace=True)
        df_merged = pd.merge(left=df,
                             right=df_gadm_1,
                             how="left",
                             left_on="gid_1",
                             right_on="gid_1")
        df_merged = df_merged.reindex(sorted(df_merged.columns), axis=1)
        df_merged_csv = df_merged.set_index("gid_1")
    
    
    output_file_path_ec2 = "{}/aqueduct30_{}_V{:02.0f}.csv".format(ec2_output_path,geographic_scale,OUTPUT_VERSION)
    df_merged_csv.to_csv(path_or_buf=output_file_path_ec2,index=True)
    
    destination_table = "{}.aqueduct30_{}_V{:02.0f}".format(BQ_DATASET_NAME,geographic_scale,OUTPUT_VERSION)

    df_merged.to_gbq(destination_table=destination_table,
                     project_id=BQ_PROJECT_ID,
                     if_exists="replace")

    return df_merged

    

In [15]:
sectors = ["Tot","Dom","Ind","Irr","Liv"]
#indicators = ["bws","bwd","iav","sev","gtd","drr","rfr","cfr","ucw","cep","udw","usa","rri"]
indicators = ["bws","bwd","iav","sev"]

In [16]:
df_appended_gid_1 = pd.DataFrame()
df_appended_gid_0 = pd.DataFrame()

for sector in sectors:
    for indicator in indicators:
        print("sector:" , sector , "indicator: ", indicator)
        df_weights_gid_1 = get_weights_df(sector)
        df_indicator_gid_1 = get_weighted_indicator_df(indicator)
        df_gid_1 = pd.merge(left=df_weights_gid_1,
                                   right=df_indicator_gid_1,
                                   how="inner",
                                   left_on="gid_1",
                                   right_on="gid_1")
        df_gid_0 = province_to_country(df_gid_1,sector,indicator)
        df_gid_0 = process_df(df_gid_0)
        df_gid_1 = process_df(df_gid_1)
        
        df_appended_gid_0 = df_appended_gid_0.append(df_gid_0)
        df_appended_gid_1 = df_appended_gid_1.append(df_gid_1)

sector: Tot indicator:  bws
sector: Tot indicator:  bwd
sector: Tot indicator:  iav
sector: Tot indicator:  sev
sector: Dom indicator:  bws
sector: Dom indicator:  bwd
sector: Dom indicator:  iav
sector: Dom indicator:  sev
sector: Ind indicator:  bws
sector: Ind indicator:  bwd
sector: Ind indicator:  iav
sector: Ind indicator:  sev
sector: Irr indicator:  bws
sector: Irr indicator:  bwd
sector: Irr indicator:  iav
sector: Irr indicator:  sev
sector: Liv indicator:  bws
sector: Liv indicator:  bwd
sector: Liv indicator:  iav
sector: Liv indicator:  sev


In [17]:
df_gid_0 = export_df(df_appended_gid_0,"country")
df_gid_1 = export_df(df_appended_gid_1,"province")

1it [00:03,  3.22s/it]
8it [00:46,  5.75s/it]


In [18]:
df_gid_0.shape

(4560, 10)

In [19]:
df_gid_0.head()

,cat,gid_0,indicator_name,label,name_0,score,score_ranked,sector,sum_weighted_indicator,sum_weights
0,3.0,AFG,bws,High,Afghanistan,3.798975,31.0,Tot,111663.715091,29393.117023
1,1.0,AGO,bws,Low - Medium,Angola,1.661512,85.0,Tot,2132.968020,1283.751460
2,0.0,ALA,bws,Low,Åland,0.460932,145.0,Tot,10.529361,22.843640
3,3.0,ALB,bws,High,Albania,3.527016,37.0,Tot,12000.282260,3402.389722
4,3.0,AND,bws,High,Andorra,3.917308,25.0,Tot,402.256135,102.686891


In [20]:
df_gid_1.shape

(72320, 12)

In [21]:
df_gid_1.head()

,cat,gid_0,gid_1,indicator_name,label,name_0,name_1,score,score_ranked,sector,sum_weighted_indicator,sum_weights
0,1.0,BRA,BRA.19_1,bws,Low - Medium,Brazil,Rio de Janeiro,1.613728,1423.0,Tot,4058.518346,2514.995991
1,0.0,BRA,BRA.13_1,bws,Low,Brazil,Minas Gerais,0.506145,2041.0,Tot,2765.254934,5463.364571
2,1.0,BRA,BRA.25_1,bws,Low - Medium,Brazil,São Paulo,1.087366,1675.0,Tot,9952.655191,9152.996072
3,1.0,BRA,BRA.8_1,bws,Low - Medium,Brazil,Espírito Santo,1.117165,1658.0,Tot,1585.571265,1419.280556
4,0.0,BRA,BRA.5_1,bws,Low,Brazil,Bahia,0.795505,1841.0,Tot,3338.599288,4196.830095


In [22]:
!aws s3 cp {ec2_output_path} {s3_output_path} --recursive

upload: ../../../../data/Y2019M01D28_RH_GA_Zonal_Stats_Table_V01/output_V06/aqueduct30_country_V06.csv to s3://wri-projects/Aqueduct30/processData/Y2019M01D28_RH_GA_Zonal_Stats_Table_V01/output_V06/aqueduct30_country_V06.csv
upload: ../../../../data/Y2019M01D28_RH_GA_Zonal_Stats_Table_V01/output_V06/aqueduct30_province_V06.csv to s3://wri-projects/Aqueduct30/processData/Y2019M01D28_RH_GA_Zonal_Stats_Table_V01/output_V06/aqueduct30_province_V06.csv


In [23]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:01:17.641749


Previous runs:   
0:00:37.409272
